## Counting the word in a Feed

In [1]:
import feedparser
import re

In [ ]:
#返回一个rss订阅源的标题 和包含的单词计数情况的字典
def getwordcount(url):
    #解析订阅源
    d = feedparser.parse(url)
    wc = {}
    
    #循环遍历所有的文章条目
    for e in d.entries:
        if 'summary' in e: summary = e.summary
        else : summary = e.description
        
        #提取一个单词列表
        words = getwords(e.title + '' + summary)
        for word in words:
            wc.setdefault(word,0)
            wc[word] +=1
return 

